In this notebook we will take our basic data set and add a lot of the features that were discussed during our EDA.

This includes:
1) A column "hashtags" which lists the number of hashtags used in total between the title and the description
2) Columns comm_to_views and likes_to_views which give the comments:views and likes:views ratios
3) A column "popular_brands" which is 1 (True) if a popular brand is mentioned and 0 (False) otherwise
4) Affiliate Columns that tell if a post has an affiliate link or discount code, as well as buisiness inquiries.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"C:\Users\arubi\Desktop\datascience2025\erdos-uncreatives\Data Cleaning and Visualization\cleaner_transformed_data_Rachael_with_no_spaces_in_column_names.csv")
df.columns

Index(['Unnamed: 0', 'channelDescription', 'channelJoinedDate',
       'channelTotalVideos', 'channelTotalViews', 'channelUsername',
       'commentsCount', 'date', 'duration', 'id', 'isChannelVerified', 'likes',
       'numberOfSubscribers', 'order', 'text', 'title', 'url', 'viewCount',
       'likes per subscriber', 'comments per subscriber',
       'views per subscriber', 'duration in seconds', 'datetime date'],
      dtype='object')

In [7]:
#Creating the hashtags column 
df["text"] = df["text"].fillna("")
df["hashtags"] = df["title"].str.count("#") + df["text"].str.count("#")

0        0
1        0
2        0
3        0
4        0
        ..
17601    5
17602    6
17603    7
17604    5
17605    7
Name: hashtags, Length: 17606, dtype: int64

In [9]:
#Creating the ratio columns

df["comm_to_views"] = df["commentsCount"] / df["viewCount"]
df["likes_to_views"] = df["likes"] / df["viewCount"]

In [23]:
#Creating the popular_brand colummn

mentions = ["natasha denona", "natashadenona", "denona", "tower 28", "tower28", "pat mcgrath", "pmg labs", "mcgrath", "patmcgrath"]
mentions += ["urban decay", "urbandecay", "colourpop", "colorpop", "colour pop", "fenty", "e.l.f.", "elf", "nyx", "essence", "benefit"]
mentions += ["anastasia", "abh", "tarte", "milk", "maybelline", "oden's eye", "oden'seye", "odenseye", "the ordinary", "theordinary"]
mentions += ["beauty of josean", "josean", "bubble", "paula's choice", "paula'schoice", "paulaschoice", "cerave", "good molecules"]
mentions += ["cosrx", "olive young", "oliveyoung", "grossman", "skinfix", "drunk elephant", "drunkelephant", "roche-posay", "roche posay", "rocheposay"]
mentions += ["supergoop", "glow recipe", "glowrecipe", "rhode"]

new_array = df["title"] + df["text"] #Creating a single array of both title and description so that my for loop works in the next step. 

popb_list = []
for item in new_array:
    item = item.lower()

    ment_bool = 0
    for ment in mentions:
        if (ment in item):
            ment_bool = 1

    popb_list.append(ment_bool) 
    
df["popular_brand"] = np.array(popb_list) 

(2801, 27)

In [25]:
#Adding affiliate columns; this is copy-pasted from Yasaman's notebook

import re
# affiliate links and discount codes
affiliate_patterns = [
    r'aff(iliate)?[ -]?link',
    r'ref(erral)?[ -]?link',
    r'partner[ -]?link',
    r'sponsored[ -]?link',
    r'^http(s)?:\/\/(www\.)?(amzn\.to|amazon\.[a-z\.]+\/[^\s]+tag=)',
    r'go\.magik\.ly',
    r'liketoknow\.it',
    r'prf\.hn',
    r'shareasale',
    r'rewardstyle',
    r'linktr\.ee'
]

discount_patterns = [
    r'disc(ount)?[ -]?code',
    r'coupon[ -]?code',
    r'promo[ -]?code',
    r'save \d+%',
    r'\d+%[ -]?off',
    r'use code[: ][a-z0-9_]+'
]


business_patterns = [
    r'business inquir(y|ies)',
    r'collaborations?',
    r'sponsorships?',
    r'partnerships?',
    r'for business',
    r'contact(\s+me)?(\s+for)?(\s+business)?'
]

def has_affiliate_or_discount(text):
    if pd.isna(text):
        return False

    has_affiliate = any(re.search(pattern, text, re.IGNORECASE) for pattern in affiliate_patterns)

    has_discount = any(re.search(pattern, text, re.IGNORECASE) for pattern in discount_patterns)

    return has_affiliate or has_discount

def has_business_inquiry(text):
    if pd.isna(text):
        return False

    return any(re.search(pattern, text, re.IGNORECASE) for pattern in business_patterns)

df['has_title_affiliate'] = df['title'].apply(has_affiliate_or_discount)
df['has_description_affiliate'] = df['text'].apply(has_affiliate_or_discount)
df['has_channel_description_affiliate'] = df['channelDescription'].apply(has_affiliate_or_discount)
df['has_any_affiliate'] = df['has_title_affiliate'] | df['has_description_affiliate'] | df['has_channel_description_affiliate']
df['has_business_inquiry'] = df['channelDescription'].apply(has_business_inquiry)

In [29]:
df['has_title_affiliate'] = df['has_title_affiliate'].astype(int)
df['has_description_affiliate'] = df['has_description_affiliate'].astype(int)
df['has_channel_description_affiliate'] = df['has_channel_description_affiliate'].astype(int)
df['has_any_affiliate'] = df['has_any_affiliate'].astype(int)
df['has_business_inquiry'] = df['has_business_inquiry'].astype(int)

In [35]:
df.to_csv("data_no_spaces_all_features.csv")